# Dummy Data

**Raihana Fawaz (1103210102)**

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [19]:
# Membuat Data Dummy untuk Klasifikasi data
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Praproses Data
scaler = StandardScaler()
X = scaler.fit_transform(X)  # Normalisasi features

# Membagi Data Menjadi Set Pelatihan dan Pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Mengonversi Data ke Tensors PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [21]:
# Menentukan model MLP
def create_mlp(input_size, hidden_layers, hidden_neurons, activation_function):
    layers = []
    # Lapisan Input ke Lapisan Tersembunyi Pertama
    layers.append(nn.Linear(input_size, hidden_neurons))
    # Menambahkan Lapisan Tersembunyi
    for _ in range(hidden_layers - 1):
        layers.append(nn.Linear(hidden_neurons, hidden_neurons))

    # Memilih Fungsi Aktivasi
    if activation_function == 'linear':
        activation = nn.Identity()
    elif activation_function == 'Sigmoid':
        activation = nn.Sigmoid()
    elif activation_function == 'ReLU':
        activation = nn.ReLU()
    elif activation_function == 'Softmax':
        activation = nn.Softmax(dim=1)
    elif activation_function == 'Tanh':
        activation = nn.Tanh()

    # Lapisan Output
    layers.append(nn.Linear(hidden_neurons, 2))  # Menghasilkan 2 Kelas (0 atau 1)

    # Menggabungkan Semua Lapisan
    model = nn.Sequential(*layers)
    return model

In [22]:
# Melatih dan Mengevaluasi Model
def train_and_evaluate(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size, epochs, learning_rate):
    # DataLoader untuk pelatihan dalam batch
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Loop pelatihan
    model.train()  # Mengatur model ke mode pelatihan
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()  # Mengatur ulang gradien
            output = model(X_batch)  # Proses forward
            loss = criterion(output, y_batch)  # Menghitung loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Memperbarui parameter model

    # Evaluasi setelah pelatihan
    model.eval()  # Mengatur model ke mode evaluasi
    with torch.no_grad():  # Menonaktifkan perhitungan gradien untuk efisiensi
        # Akurasi pelatihan
        train_output = model(X_train_tensor)
        train_pred = torch.argmax(train_output, dim=1)  # Mendapatkan kelas yang diprediksi
        train_accuracy = accuracy_score(y_train_tensor.numpy(), train_pred.numpy())  # Menghitung akurasi pelatihan
        train_loss = criterion(train_output, y_train_tensor).item()  # Menghitung loss pelatihan

        # Akurasi pengujian
        test_output = model(X_test_tensor)
        test_pred = torch.argmax(test_output, dim=1)  # Mendapatkan kelas yang diprediksi
        test_accuracy = accuracy_score(y_test_tensor.numpy(), test_pred.numpy())  # Menghitung akurasi pengujian
        test_loss = criterion(test_output, y_test_tensor).item()  # Menghitung loss pengujian

    return train_accuracy, train_loss, test_accuracy, test_loss  # Mengembalikan hasil

In [35]:
# Langkah 5: Bereksperimen dengan Hyperparameter
hidden_layers_options = [1, 2, 3]  # Jumlah lapisan tersembunyi
hidden_neurons_options = [4, 8, 16, 32, 64]  # Jumlah neuron pada setiap lapisan tersembunyi
activation_functions = ['linear', 'Sigmoid', 'ReLU', 'Softmax', 'Tanh']  # Fungsi aktivasi
epochs_options = [25, 50, 100, 250]  # Jumlah epoch
learning_rates = [0.1, 0.01, 0.001, 0.0001]  # Tingkat pembelajaran
batch_sizes = [64, 128, 256, 512]  # Ukuran batch

# Melacak hasil eksperimen
results = []  # Untuk menyimpan hasil dari setiap eksperimen
best_result = None
worst_result = None
best_per_activation = {activation: None for activation in activation_functions}  # Melacak hasil terbaik untuk setiap fungsi aktivasi

# Loop melalui semua kombinasi hyperparameter
for hidden_layers in hidden_layers_options:
    for hidden_neurons in hidden_neurons_options:
        for activation_function in activation_functions:
            for epochs in epochs_options:
                for learning_rate in learning_rates:
                    for batch_size in batch_sizes:
                        # Mencetak konfigurasi eksperimen saat ini
                        print(f"Melatih dengan {hidden_layers} lapisan tersembunyi, {hidden_neurons} neuron, fungsi aktivasi {activation_function}, "
                              f"{epochs} epoch, tingkat pembelajaran {learning_rate}, ukuran batch {batch_size}")

                        # Membuat model untuk setiap kombinasi hyperparameter
                        model = create_mlp(X_train_tensor.shape[1], hidden_layers, hidden_neurons, activation_function)

                        # Melatih dan mengevaluasi model
                        train_acc, train_loss, test_acc, test_loss = train_and_evaluate(
                            model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size, epochs, learning_rate
                        )

                        # Menyimpan hasil
                        results.append({
                            'Hidden Layers': hidden_layers,
                            'Hidden Neurons': hidden_neurons,
                            'Activation Function': activation_function,
                            'Epochs': epochs,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size,
                            'Train Accuracy': train_acc,
                            'Train Loss': train_loss,
                            'Test Accuracy': test_acc,
                            'Test Loss': test_loss
                        })

                        # Mencetak hasil akhir dari eksperimen ini
                        print(f"Akurasi Pelatihan: {train_acc * 100:.2f}%")
                        print(f"Loss Pelatihan: {train_loss:.4f}")
                        print(f"Akurasi Pengujian: {test_acc * 100:.2f}%")
                        print(f"Loss Pengujian: {test_loss:.4f}\n")

                        # Memperbarui hasil terbaik dan terburuk
                        if best_result is None or test_acc > best_result['Test Accuracy']:
                            best_result = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }
                        if worst_result is None or test_acc < worst_result['Test Accuracy']:
                            worst_result = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }

                        # Melacak konfigurasi terbaik untuk setiap fungsi aktivasi
                        if best_per_activation[activation_function] is None or test_acc > best_per_activation[activation_function]['Test Accuracy']:
                            best_per_activation[activation_function] = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }


Streaming output truncated to the last 5000 lines.
Loss Pengujian: 0.3831

Melatih dengan 3 lapisan tersembunyi, 16 neuron, fungsi aktivasi Sigmoid, 250 epoch, tingkat pembelajaran 0.0001, ukuran batch 512
Akurasi Pelatihan: 87.86%
Loss Pelatihan: 0.3860
Akurasi Pengujian: 84.00%
Loss Pengujian: 0.4308

Melatih dengan 3 lapisan tersembunyi, 16 neuron, fungsi aktivasi ReLU, 25 epoch, tingkat pembelajaran 0.1, ukuran batch 64
Akurasi Pelatihan: 88.57%
Loss Pelatihan: 0.3261
Akurasi Pengujian: 84.00%
Loss Pengujian: 0.4276

Melatih dengan 3 lapisan tersembunyi, 16 neuron, fungsi aktivasi ReLU, 25 epoch, tingkat pembelajaran 0.1, ukuran batch 128
Akurasi Pelatihan: 89.14%
Loss Pelatihan: 0.3298
Akurasi Pengujian: 85.33%
Loss Pengujian: 0.4253

Melatih dengan 3 lapisan tersembunyi, 16 neuron, fungsi aktivasi ReLU, 25 epoch, tingkat pembelajaran 0.1, ukuran batch 256
Akurasi Pelatihan: 88.57%
Loss Pelatihan: 0.3104
Akurasi Pengujian: 85.67%
Loss Pengujian: 0.3871

Melatih dengan 3 lapisan te

In [36]:
# Menampilkan Konfigurasi Hyperparameter Terbaik dan Terburuk
print("\nBest Hyperparameter Configuration (Overall):")
print(f"Hidden Layers: {best_result['Hyperparameters']['Hidden Layers']}")
print(f"Hidden Neurons: {best_result['Hyperparameters']['Hidden Neurons']}")
print(f"Activation Function: {best_result['Hyperparameters']['Activation Function']}")
print(f"Epochs: {best_result['Hyperparameters']['Epochs']}")
print(f"Learning Rate: {best_result['Hyperparameters']['Learning Rate']}")
print(f"Batch Size: {best_result['Hyperparameters']['Batch Size']}")
print(f"Test Accuracy: {best_result['Test Accuracy'] * 100:.2f}%")


Best Hyperparameter Configuration (Overall):
Hidden Layers: 2
Hidden Neurons: 16
Activation Function: Softmax
Epochs: 100
Learning Rate: 0.1
Batch Size: 128
Test Accuracy: 87.67%


In [37]:
print("\nWorst Hyperparameter Configuration:")
print(f"Hidden Layers: {worst_result['Hyperparameters']['Hidden Layers']}")
print(f"Hidden Neurons: {worst_result['Hyperparameters']['Hidden Neurons']}")
print(f"Activation Function: {worst_result['Hyperparameters']['Activation Function']}")
print(f"Epochs: {worst_result['Hyperparameters']['Epochs']}")
print(f"Learning Rate: {worst_result['Hyperparameters']['Learning Rate']}")
print(f"Batch Size: {worst_result['Hyperparameters']['Batch Size']}")
print(f"Test Accuracy: {worst_result['Test Accuracy'] * 100:.2f}%")


Worst Hyperparameter Configuration:
Hidden Layers: 1
Hidden Neurons: 8
Activation Function: ReLU
Epochs: 25
Learning Rate: 0.0001
Batch Size: 512
Test Accuracy: 30.00%


In [39]:
# Menampilkan Konfigurasi Terbaik untuk Setiap Fungsi Aktivasi
for activation in activation_functions:
    print(f"\nBest Configuration for Activation Function {activation}:")
    if best_per_activation[activation]:
        print(f"Hidden Layers: {best_per_activation[activation]['Hyperparameters']['Hidden Layers']}")
        print(f"Hidden Neurons: {best_per_activation[activation]['Hyperparameters']['Hidden Neurons']}")
        print(f"Activation Function: {activation}")
        print(f"Epochs: {best_per_activation[activation]['Hyperparameters']['Epochs']}")
        print(f"Learning Rate: {best_per_activation[activation]['Hyperparameters']['Learning Rate']}")
        print(f"Batch Size: {best_per_activation[activation]['Hyperparameters']['Batch Size']}")
        print(f"Test Accuracy: {best_per_activation[activation]['Test Accuracy'] * 100:.2f}%")
    else:
        print("No result for this activation function.")


Best Configuration for Activation Function linear:
Hidden Layers: 2
Hidden Neurons: 64
Activation Function: linear
Epochs: 250
Learning Rate: 0.1
Batch Size: 256
Test Accuracy: 87.33%

Best Configuration for Activation Function Sigmoid:
Hidden Layers: 3
Hidden Neurons: 8
Activation Function: Sigmoid
Epochs: 50
Learning Rate: 0.01
Batch Size: 128
Test Accuracy: 87.00%

Best Configuration for Activation Function ReLU:
Hidden Layers: 1
Hidden Neurons: 8
Activation Function: ReLU
Epochs: 25
Learning Rate: 0.1
Batch Size: 512
Test Accuracy: 87.00%

Best Configuration for Activation Function Softmax:
Hidden Layers: 2
Hidden Neurons: 16
Activation Function: Softmax
Epochs: 100
Learning Rate: 0.1
Batch Size: 128
Test Accuracy: 87.67%

Best Configuration for Activation Function Tanh:
Hidden Layers: 2
Hidden Neurons: 16
Activation Function: Tanh
Epochs: 25
Learning Rate: 0.01
Batch Size: 64
Test Accuracy: 87.33%
